In [1]:
import pandas as pd
import numpy as np
from itertools import chain
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
def count_vectorizer(sentences, params={}):
    default_params = {'strip_accents': None, 
                    'lowercase': True,
                    'preprocessor': None, 
                    'tokenizer': None, 
                    'stop_words': None, 
                    'ngram_range': (1, 1), 
                    'analyzer': 'word', 
                    'max_df': 1.0, 
                    'min_df': 1, 
                    'max_features': None, 
                    'vocabulary': None}
    default_params.update(params)
    
    cv = CountVectorizer(sentences, **default_params)
    trans_sent = cv.fit_transform(sentences)
    
    return cv, trans_sent

In [3]:
df = pd.read_excel("../Data/Market_Manipulation/2_enron_data_with_stop.xlsx", nrows=200)
df.dropna(inplace=True)
df.head()

,id,processed_text
0,0,here is our forecast
1,1,traveling to have business meeting takes the f...
2,2,test successful . way to go
3,3,randy can you send me schedule of the salary a...
4,5,greg how about either next tuesday or thursday...


In [10]:
def custom_tokenizer(para):
    words = list()
    for sent in para.split(' . '):
        words.append(sent.split())
    return list(chain(*words))

In [13]:
cv_obj, cv_sent = count_vectorizer(df['processed_text'].tolist(), {'tokenizer': custom_tokenizer})